In [7]:
import pandas as pd

# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Ruta al archivo en Google Drive
file_path = '/content/drive/MyDrive/nav/Caudales-medios-mensuales-Rio-Itata.xlsx'

# Cargar el archivo Excel
excel = pd.ExcelFile(file_path)

# Obtener los nombres de las hojas
sheets = excel.sheet_names

# Cargar todas las hojas en dataframes
dfs = [excel.parse(sheet) for sheet in sheets]

# Inicializar un diccionario para almacenar la suma de datos por año
yearly_data = {
    2019: 0,
    2020: 0,
    2021: 0,
    2022: 0
}

# Definir las columnas que contienen los datos mensuales
month_columns = ['Unnamed: 3', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 9',
                 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15',
                 'Unnamed: 16', 'Unnamed: 17']

# Función para extraer y sumar los datos mensuales de un dataframe
def sum_monthly_data(df):
    # Identificar las filas que contienen datos anuales
    df_filtered = df.dropna(how='all').reset_index(drop=True)

    if df_filtered.shape[0] > 1:
        for index, row in df_filtered.iterrows():
            if row['GOBIERNO DE CHILE'] in [2019, 2020, 2021, 2022]:
                year = row['GOBIERNO DE CHILE']
                monthly_values = pd.to_numeric(row[month_columns], errors='coerce')  # Convertir a numérico, forzando errores a NaN
                yearly_sum = monthly_values.sum(skipna=True)  # Sumar los valores numéricos, ignorando NaNs
                yearly_data[year] += yearly_sum

# Procesar cada dataframe
for df in dfs:
    sum_monthly_data(df)

# Convertir los resultados a un DataFrame para visualización
yearly_data_df = pd.DataFrame.from_dict(yearly_data, orient='index', columns=['Caudal (m3/s)'])
yearly_data_df.index.name = 'Año'

# Añadir un título al DataFrame
title = "Caudales Medios Anuales Río Itata (m3/s)"
yearly_data_df.columns.name = title

# Guardar el resultado en un archivo CSV
csv_file_path = '/content/drive/MyDrive/nav/Yearly_Data_Summary.csv'
yearly_data_df.to_csv(csv_file_path, index_label='Año')

# Guardar el resultado en un archivo XLSX
xlsx_file_path = '/content/drive/MyDrive/nav/Yearly_Data_Summary.xlsx'
with pd.ExcelWriter(xlsx_file_path) as writer:
    yearly_data_df.to_excel(writer, index_label='Año', sheet_name='Resumen')

print(f'Resultados guardados en {csv_file_path} y {xlsx_file_path}')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Resultados guardados en /content/drive/MyDrive/nav/Yearly_Data_Summary.csv y /content/drive/MyDrive/nav/Yearly_Data_Summary.xlsx
